## Gerador de Horários de Ponto para Jornada de 8 Horas Diárias

- Por: Luciana Sampaio
- Data: 03/04/25 

-----

- **4 registros por dia:** Cada dia contém *quatro horários de marcação de ponto*: **Entrada**, **Saída para Almoço**, **Retorno do Almoço** e **Saída Final**. Assim, há dois intervalos principais (manhã e tarde) para cada dia trabalhado.

- **Jornada total de 8 horas:** O gerador calcula os horários de forma que a **soma do tempo trabalhado** (manhã + tarde, excluindo o intervalo de almoço) seja de **8 horas**. Ou seja, a duração do trabalho pela manhã somada à da tarde totaliza 8h. O intervalo de almoço não conta como tempo trabalhado.

- **Variações aleatórias nos horários:** Para evitar dias idênticos, são introduzidas **pequenas variações aleatórias de 1 a 10 minutos** nos horários de entrada, saída e retorno. Com isso, cada dia terá horários ligeiramente diferentes (por exemplo, entrada às 08:05 em um dia e 08:12 no outro), mantendo a naturalidade.

- **Cálculo do horário de saída:** O horário de saída final é calculado com base na entrada e na duração do almoço para garantir as 8 horas de trabalho. Por exemplo, se alguém entra às 08:30 e faz 1h de almoço, deverá sair aproximadamente às 17:30. Pequenas diferenças de minutos podem ocorrer dependendo das variações aleatórias (por exemplo, saída às 17:34 ou 17:21 em vez de 17:30, gerando 8h02 ou 7h59 de trabalho, que podem ser contabilizados no banco de horas).


In [10]:
import pandas as pd
import random
from datetime import datetime, timedelta
import unicodedata


# 🔸 Função para remover acentos de qualquer string
def remover_acentos(texto):
    if isinstance(texto, str):
        return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    return texto


def gerar_minuto_valido(minutos_existentes):
    minutos_possiveis = [i for i in range(0, 60) if str(i)[-1] != '1' and i not in minutos_existentes]
    if not minutos_possiveis:
        raise ValueError("Nao ha minutos disponiveis que atendam as regras.")
    minuto = random.choice(minutos_possiveis)
    minutos_existentes.append(minuto)
    return minuto


def gerar_horarios_dia(data_referencia):
    minutos_usados = []

    hora_base = datetime.combine(data_referencia, datetime.strptime("08:00", "%H:%M").time())
    incremento = random.randint(0, 75)

    entrada_candidata = hora_base + timedelta(minutes=incremento)
    entrada = entrada_candidata.replace(minute=gerar_minuto_valido(minutos_usados))

    duracao_manha = random.randint(180, 240)
    saida_almoco = entrada + timedelta(minutes=duracao_manha)
    saida_almoco = saida_almoco.replace(minute=gerar_minuto_valido(minutos_usados))

    intervalo_almoco = random.randint(60, 90)
    retorno_almoco = saida_almoco + timedelta(minutes=intervalo_almoco)
    retorno_almoco = retorno_almoco.replace(minute=gerar_minuto_valido(minutos_usados))

    duracao_tarde = 480 - duracao_manha
    saida_final = retorno_almoco + timedelta(minutes=duracao_tarde)
    saida_final = saida_final.replace(minute=gerar_minuto_valido(minutos_usados))

    total_trabalhado = timedelta(minutes=duracao_manha + duracao_tarde)

    observacao = "" if random.random() > 0.1 else "Registro inserido manualmente"

    return {
        "Data": data_referencia.strftime("%d/%m/%Y"),
        "Entrada_1": entrada.strftime("%H:%M"),
        "Saida_1": saida_almoco.strftime("%H:%M"),
        "Entrada_2": retorno_almoco.strftime("%H:%M"),
        "Saida_2": saida_final.strftime("%H:%M"),
        "Banco_de_Horas": f"{total_trabalhado.seconds // 3600:02d}:{(total_trabalhado.seconds % 3600) // 60:02d}",
        "Observacao": remover_acentos(observacao)
    }


def gerar_datas_uteis(inicio, quantidade):
    datas = []
    data_atual = inicio
    while len(datas) < quantidade:
        if data_atual.weekday() < 5:
            datas.append(data_atual)
        data_atual += timedelta(days=1)
    return datas


# 📅 Data inicial
data_inicial = datetime.today()

# 📅 Gerar 30 dias uteis
datas_uteis = gerar_datas_uteis(data_inicial, 30)

# 🏗️ Gerar os dados
dados = [gerar_horarios_dia(data) for data in datas_uteis]

# 📑 Criar DataFrame
df = pd.DataFrame(dados)

# 👀 Verificar
print(df)

# 💾 Salvar CSV sem acentos
df.to_csv("ponto_simulado_final.csv", sep=";", index=False, encoding="utf-8")


          Data Entrada_1 Saida_1 Entrada_2 Saida_2 Banco_de_Horas  \
0   30/05/2025     09:15   13:14     14:33   18:08          08:00   
1   02/06/2025     08:33   11:04     12:46   17:28          08:00   
2   03/06/2025     08:47   12:40     13:58   18:55          08:00   
3   04/06/2025     08:26   11:02     12:19   17:03          08:00   
4   05/06/2025     09:49   13:15     14:05   18:25          08:00   
5   06/06/2025     09:47   13:36     14:37   19:49          08:00   
6   09/06/2025     09:39   12:08     13:49   18:18          08:00   
7   10/06/2025     08:47   12:25     13:15   17:09          08:00   
8   11/06/2025     08:50   12:37     13:02   17:10          08:00   
9   12/06/2025     09:14   13:46     15:23   19:50          08:00   
10  13/06/2025     09:10   12:58     14:07   18:44          08:00   
11  16/06/2025     09:18   12:43     13:08   18:19          08:00   
12  17/06/2025     09:00   12:04     13:42   18:14          08:00   
13  18/06/2025     09:49   13:46  